# Day 3 - Gradio UI Integration with LangGraph
# Gradio를 통한 사용자 인터페이스 래핑

이 실습에서는 앞서 구축한 LangGraph 시스템들을 Gradio를 사용하여
사용자 친화적인 웹 인터페이스로 래핑합니다.

## 핵심 개념
- Gradio 기본 사용법
- 다중 탭 인터페이스 구성
- 실시간 스트리밍 출력
- 파일 업로드/다운로드
- 사용자 세션 관리

## 통합할 시스템들
- 고급 RAG 시스템
- Text2SQL 시스템
- MCP 통합 시스템

In [ ]:
!pip install gradio langchain langgraph transformers torch chromadb sentence-transformers sqlite3 pandas yfinance wikipedia-api

In [ ]:
import os
import sqlite3
import json
import tempfile
import time
from typing import List, Dict, Any, Optional, Tuple, Iterator
from datetime import datetime
import threading
import queue

import gradio as gr
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# LangGraph 관련 임포트
from langchain.schema import Document
from langgraph.graph import StateGraph, END

print("라이브러리 import 완료")

## 1. Day 1 파인튜닝 모델 및 시스템 초기화

In [ ]:
class Day1FinetunedLLM:
    def __init__(self, model_name="ryanu/my-exaone-raft-model"):
        self.model_name = model_name
        self.base_model = "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct"
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        print(f"모델 로딩 중: {model_name}")
        print(f"사용 디바이스: {self.device}")
        
        try:
            # 토크나이저 로드
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.base_model,
                trust_remote_code=True
            )
            
            # 베이스 모델 로드
            self.model = AutoModelForCausalLM.from_pretrained(
                self.base_model,
                torch_dtype=torch.float16,
                device_map="auto",
                trust_remote_code=True
            )
            
            # 파인튜닝된 어댑터 로드
            try:
                self.model = PeftModel.from_pretrained(self.model, model_name)
                print("✅ 파인튜닝 모델 로드 완료")
            except Exception as e:
                print(f"⚠️ 파인튜닝 모델 로드 실패, 베이스 모델 사용: {e}")
                
        except Exception as e:
            print(f"⚠️ 모델 로드 실패: {e}")
            # 데모용으로 더미 모델 사용
            self.model = None
            self.tokenizer = None
    
    def generate(self, prompt: str, max_length: int = 512, temperature: float = 0.7) -> str:
        """텍스트 생성"""
        if self.model is None:
            # 데모용 더미 응답
            return f"[데모 모드] 질문 '{prompt[:50]}...'에 대한 응답입니다."
        
        try:
            inputs = self.tokenizer(
                prompt, 
                return_tensors="pt", 
                truncation=True, 
                max_length=2048
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=inputs['input_ids'].shape[1] + max_length,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            generated_text = self.tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:], 
                skip_special_tokens=True
            )
            
            return generated_text.strip()
            
        except Exception as e:
            return f"생성 오류: {str(e)}"

# 글로벌 모델 인스턴스
llm = Day1FinetunedLLM()
print("✅ LLM 초기화 완료")

## 2. 간소화된 RAG 시스템

In [ ]:
class SimpleRAGSystem:
    """Gradio UI용 간소화된 RAG 시스템"""
    
    def __init__(self):
        self.documents = []
        self._create_sample_documents()
    
    def _create_sample_documents(self):
        """샘플 문서 생성"""
        sample_docs = [
            "머신러닝은 인공지능의 하위 분야로, 컴퓨터가 명시적으로 프로그래밍되지 않고도 학습할 수 있게 하는 기술입니다.",
            "딥러닝은 인공 신경망을 사용하는 머신러닝의 특별한 형태입니다. CNN, RNN, 트랜스포머 등 다양한 아키텍처가 있습니다.",
            "자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 생성할 수 있게 하는 AI 분야입니다.",
            "트랜스포머는 어텐션 메커니즘만을 사용하는 신경망 아키텍처로, BERT, GPT 등의 기초가 되었습니다.",
            "파인튜닝은 사전 훈련된 모델을 특정 작업에 맞게 추가로 훈련시키는 기법입니다.",
            "RAG(Retrieval-Augmented Generation)는 외부 지식을 검색하여 생성 모델에 제공하는 기법입니다.",
            "LangChain은 언어 모델을 활용한 애플리케이션 개발을 위한 프레임워크입니다.",
            "벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데이터베이스입니다."
        ]
        
        for i, doc in enumerate(sample_docs):
            self.documents.append({
                "id": i,
                "content": doc,
                "metadata": {"topic": "AI", "source": "sample"}
            })
    
    def add_document(self, content: str, metadata: Dict = None):
        """문서 추가"""
        doc_id = len(self.documents)
        self.documents.append({
            "id": doc_id,
            "content": content,
            "metadata": metadata or {}
        })
        return doc_id
    
    def search_documents(self, query: str, top_k: int = 3) -> List[Dict]:
        """간단한 키워드 기반 문서 검색"""
        query_lower = query.lower()
        results = []
        
        for doc in self.documents:
            content_lower = doc["content"].lower()
            # 간단한 키워드 매칭 점수
            score = 0
            for word in query_lower.split():
                if word in content_lower:
                    score += content_lower.count(word)
            
            if score > 0:
                results.append({
                    "document": doc,
                    "score": score
                })
        
        # 점수 기준 정렬 후 상위 k개 반환
        results.sort(key=lambda x: x["score"], reverse=True)
        return results[:top_k]
    
    def generate_answer(self, query: str, progress_callback=None) -> Tuple[str, List[Dict]]:
        """RAG 답변 생성"""
        if progress_callback:
            progress_callback("🔍 문서 검색 중...")
        
        # 관련 문서 검색
        search_results = self.search_documents(query)
        
        if not search_results:
            return "관련 문서를 찾을 수 없습니다.", []
        
        if progress_callback:
            progress_callback(f"📚 {len(search_results)}개 문서 발견")
        
        # 컨텍스트 구성
        context_parts = []
        for i, result in enumerate(search_results, 1):
            doc = result["document"]
            context_parts.append(f"[참고자료 {i}] {doc['content']}")
        
        context = "\n\n".join(context_parts)
        
        if progress_callback:
            progress_callback("🤖 답변 생성 중...")
        
        # 답변 생성 프롬프트
        prompt = f"""다음 참고자료를 바탕으로 질문에 답변해주세요.

참고자료:
{context}

질문: {query}

답변:
- 참고자료의 내용을 바탕으로 정확하고 구체적으로 답변해주세요
- 참고자료에 없는 내용은 추측하지 마세요

답변:"""
        
        answer = llm.generate(prompt, max_length=400, temperature=0.3)
        
        if progress_callback:
            progress_callback("✅ 답변 생성 완료")
        
        return answer, [r["document"] for r in search_results]

# RAG 시스템 초기화
rag_system = SimpleRAGSystem()
print("✅ RAG 시스템 초기화 완료")

## 3. 간소화된 Text2SQL 시스템

In [ ]:
class SimpleText2SQLSystem:
    """Gradio UI용 간소화된 Text2SQL 시스템"""
    
    def __init__(self):
        self.db_path = self._create_sample_database()
        self.schema_info = self._get_schema_info()
    
    def _create_sample_database(self) -> str:
        """샘플 데이터베이스 생성"""
        # 임시 데이터베이스 파일 생성
        db_path = tempfile.mktemp(suffix=".db")
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # 간단한 테이블 생성
        cursor.execute('''
            CREATE TABLE products (
                id INTEGER PRIMARY KEY,
                name TEXT,
                price REAL,
                category TEXT
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE orders (
                id INTEGER PRIMARY KEY,
                product_id INTEGER,
                quantity INTEGER,
                order_date TEXT,
                FOREIGN KEY (product_id) REFERENCES products (id)
            )
        ''')
        
        # 샘플 데이터 삽입
        products = [
            (1, '노트북', 1200000, '전자제품'),
            (2, '마우스', 30000, '전자제품'),
            (3, '키보드', 80000, '전자제품'),
            (4, '모니터', 400000, '전자제품'),
            (5, '의자', 200000, '가구')
        ]
        cursor.executemany('INSERT INTO products VALUES (?, ?, ?, ?)', products)
        
        orders = [
            (1, 1, 2, '2024-01-15'),
            (2, 2, 5, '2024-01-16'),
            (3, 3, 3, '2024-01-17'),
            (4, 1, 1, '2024-01-18'),
            (5, 4, 2, '2024-01-19')
        ]
        cursor.executemany('INSERT INTO orders VALUES (?, ?, ?, ?)', orders)
        
        conn.commit()
        conn.close()
        
        return db_path
    
    def _get_schema_info(self) -> str:
        """스키마 정보 가져오기"""
        return """데이터베이스 스키마:

테이블: products
- id: INTEGER (기본키)
- name: TEXT (상품명)
- price: REAL (가격)
- category: TEXT (카테고리)

테이블: orders
- id: INTEGER (기본키)
- product_id: INTEGER (상품 ID, products.id 참조)
- quantity: INTEGER (수량)
- order_date: TEXT (주문 날짜)

관계: orders.product_id → products.id"""
    
    def generate_sql(self, question: str, progress_callback=None) -> Tuple[str, pd.DataFrame, str]:
        """자연어 질문을 SQL로 변환하고 실행"""
        if progress_callback:
            progress_callback("🧠 질문 분석 중...")
        
        # SQL 생성 프롬프트
        prompt = f"""다음 데이터베이스 스키마를 바탕으로 자연어 질문을 SQL 쿼리로 변환해주세요.

{self.schema_info}

질문: {question}

SQLite 문법을 사용하여 SQL 쿼리만 생성해주세요.

SQL:"""
        
        if progress_callback:
            progress_callback("🔄 SQL 생성 중...")
        
        sql_query = llm.generate(prompt, max_length=200, temperature=0.1)
        
        # SQL 정제
        sql_query = sql_query.split('\n')[0].strip()
        if sql_query.endswith(';'):
            sql_query = sql_query[:-1]
        
        if progress_callback:
            progress_callback("⚡ SQL 실행 중...")
        
        # SQL 실행
        try:
            conn = sqlite3.connect(self.db_path)
            result_df = pd.read_sql_query(sql_query, conn)
            conn.close()
            
            if progress_callback:
                progress_callback(f"✅ 완료: {len(result_df)}개 행 반환")
            
            return sql_query, result_df, "성공"
            
        except Exception as e:
            error_msg = f"SQL 실행 오류: {str(e)}"
            if progress_callback:
                progress_callback(f"❌ 오류: {error_msg}")
            
            return sql_query, pd.DataFrame(), error_msg

# Text2SQL 시스템 초기화
text2sql_system = SimpleText2SQLSystem()
print("✅ Text2SQL 시스템 초기화 완료")

## 4. 간소화된 MCP 시스템

In [ ]:
class SimpleMCPSystem:
    """Gradio UI용 간소화된 MCP 시스템"""
    
    def __init__(self):
        self.tools = {
            "calculator": self._calculator,
            "weather": self._weather,
            "search": self._search
        }
    
    def _calculator(self, expression: str) -> Dict[str, Any]:
        """계산기 도구"""
        try:
            # 안전한 계산을 위해 허용된 문자만 사용
            allowed_chars = set('0123456789+-*/.() ')
            if not all(c in allowed_chars for c in expression):
                return {"success": False, "error": "허용되지 않은 문자"}
            
            result = eval(expression)
            return {
                "success": True,
                "expression": expression,
                "result": result
            }
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def _weather(self, city: str) -> Dict[str, Any]:
        """날씨 도구 (시뮬레이션)"""
        weather_data = {
            "서울": {"temp": 15, "desc": "맑음"},
            "부산": {"temp": 18, "desc": "구름많음"},
            "제주": {"temp": 20, "desc": "비"}
        }
        
        if city in weather_data:
            data = weather_data[city]
            return {
                "success": True,
                "city": city,
                "temperature": data["temp"],
                "description": data["desc"]
            }
        else:
            return {
                "success": True,
                "city": city,
                "temperature": 22,
                "description": "맑음"
            }
    
    def _search(self, query: str) -> Dict[str, Any]:
        """검색 도구 (시뮬레이션)"""
        return {
            "success": True,
            "query": query,
            "results": [
                f"{query}에 대한 검색 결과 1",
                f"{query}에 대한 검색 결과 2",
                f"{query}에 대한 검색 결과 3"
            ]
        }
    
    def process_query(self, query: str, progress_callback=None) -> str:
        """쿼리 처리"""
        if progress_callback:
            progress_callback("🧠 쿼리 분석 중...")
        
        query_lower = query.lower()
        results = []
        
        # 간단한 키워드 기반 도구 선택
        if any(word in query_lower for word in ['계산', '+', '-', '*', '/', '더하기', '빼기']):
            if progress_callback:
                progress_callback("🧮 계산기 실행 중...")
            
            # 수식 추출 (간단한 방식)
            import re
            math_pattern = r'[0-9+\-*/().\s]+'
            matches = re.findall(math_pattern, query)
            
            expression = None
            for match in matches:
                if any(op in match for op in ['+', '-', '*', '/']):
                    expression = match.strip()
                    break
            
            if expression:
                result = self._calculator(expression)
                if result["success"]:
                    results.append(f"계산 결과: {expression} = {result['result']}")
                else:
                    results.append(f"계산 오류: {result['error']}")
        
        if any(word in query_lower for word in ['날씨', 'weather']):
            if progress_callback:
                progress_callback("🌤 날씨 정보 조회 중...")
            
            cities = ['서울', '부산', '대구', '인천', '제주']
            city = "서울"  # 기본값
            
            for c in cities:
                if c in query:
                    city = c
                    break
            
            result = self._weather(city)
            if result["success"]:
                results.append(f"{city} 날씨: {result['temperature']}°C, {result['description']}")
        
        if any(word in query_lower for word in ['검색', '찾아', '알려']):
            if progress_callback:
                progress_callback("🔍 검색 중...")
            
            result = self._search(query)
            if result["success"]:
                results.append(f"검색 결과: {', '.join(result['results'])}")
        
        if not results:
            if progress_callback:
                progress_callback("💬 기본 응답 생성 중...")
            
            # 기본 응답 생성
            prompt = f"다음 질문에 간단히 답변해주세요: {query}"
            response = llm.generate(prompt, max_length=200, temperature=0.5)
            results.append(response)
        
        if progress_callback:
            progress_callback("✅ 처리 완료")
        
        return "\n\n".join(results)

# MCP 시스템 초기화
mcp_system = SimpleMCPSystem()
print("✅ MCP 시스템 초기화 완료")

## 5. 채팅 기록 관리

In [ ]:
class ChatHistory:
    """채팅 기록 관리 클래스"""
    
    def __init__(self):
        self.histories = {
            "rag": [],
            "text2sql": [],
            "mcp": []
        }
    
    def add_message(self, system: str, user_msg: str, bot_msg: str):
        """메시지 추가"""
        if system in self.histories:
            self.histories[system].append([user_msg, bot_msg])
    
    def get_history(self, system: str) -> List[List[str]]:
        """특정 시스템의 채팅 기록 반환"""
        return self.histories.get(system, [])
    
    def clear_history(self, system: str):
        """특정 시스템의 채팅 기록 삭제"""
        if system in self.histories:
            self.histories[system] = []
    
    def export_history(self, system: str) -> str:
        """채팅 기록을 텍스트로 내보내기"""
        history = self.get_history(system)
        if not history:
            return f"{system} 시스템의 채팅 기록이 없습니다."
        
        export_text = f"{system.upper()} 시스템 채팅 기록\n"
        export_text += f"내보낸 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        export_text += "=" * 50 + "\n\n"
        
        for i, (user_msg, bot_msg) in enumerate(history, 1):
            export_text += f"[{i}] 사용자: {user_msg}\n"
            export_text += f"[{i}] 시스템: {bot_msg}\n\n"
        
        return export_text

# 글로벌 채팅 기록 관리자
chat_history = ChatHistory()
print("✅ 채팅 기록 관리자 초기화 완료")

## 6. Gradio 인터페이스 함수들

In [ ]:
def rag_chat_fn(message: str, history: List[List[str]]) -> Tuple[str, List[List[str]]]:
    """RAG 채팅 함수"""
    if not message.strip():
        return "", history
    
    try:
        # 진행 상황을 위한 임시 응답
        temp_response = "🔍 문서를 검색하고 답변을 생성 중입니다..."
        
        # RAG 시스템으로 답변 생성
        answer, documents = rag_system.generate_answer(message)
        
        # 참고 문서 정보 추가
        if documents:
            doc_info = "\n\n📚 참고 문서:\n"
            for i, doc in enumerate(documents, 1):
                doc_info += f"{i}. {doc['content'][:100]}...\n"
            answer += doc_info
        
        # 채팅 기록에 추가
        chat_history.add_message("rag", message, answer)
        
        # 히스토리 업데이트
        new_history = history + [[message, answer]]
        
        return "", new_history
        
    except Exception as e:
        error_msg = f"오류가 발생했습니다: {str(e)}"
        new_history = history + [[message, error_msg]]
        return "", new_history

def text2sql_chat_fn(message: str, history: List[List[str]]) -> Tuple[str, List[List[str]]]:
    """Text2SQL 채팅 함수"""
    if not message.strip():
        return "", history
    
    try:
        # SQL 생성 및 실행
        sql_query, result_df, status = text2sql_system.generate_sql(message)
        
        # 응답 구성
        response = f"🔍 생성된 SQL:\n```sql\n{sql_query}\n```\n\n"
        
        if status == "성공":
            if not result_df.empty:
                response += f"📊 결과 ({len(result_df)}개 행):\n{result_df.to_string(index=False)}\n\n"
                
                # 결과 해석
                interpretation_prompt = f"""다음 SQL 쿼리와 결과를 바탕으로 사용자 질문에 대한 자연어 답변을 생성해주세요.
                
질문: {message}
SQL: {sql_query}
결과: {result_df.to_string()}

자연어 답변:"""
                
                interpretation = llm.generate(interpretation_prompt, max_length=200)
                response += f"💬 해석: {interpretation}"
            else:
                response += "📊 결과: 데이터가 없습니다."
        else:
            response += f"❌ 오류: {status}"
        
        # 채팅 기록에 추가
        chat_history.add_message("text2sql", message, response)
        
        # 히스토리 업데이트
        new_history = history + [[message, response]]
        
        return "", new_history
        
    except Exception as e:
        error_msg = f"오류가 발생했습니다: {str(e)}"
        new_history = history + [[message, error_msg]]
        return "", new_history

def mcp_chat_fn(message: str, history: List[List[str]]) -> Tuple[str, List[List[str]]]:
    """MCP 채팅 함수"""
    if not message.strip():
        return "", history
    
    try:
        # MCP 시스템으로 처리
        response = mcp_system.process_query(message)
        
        # 채팅 기록에 추가
        chat_history.add_message("mcp", message, response)
        
        # 히스토리 업데이트
        new_history = history + [[message, response]]
        
        return "", new_history
        
    except Exception as e:
        error_msg = f"오류가 발생했습니다: {str(e)}"
        new_history = history + [[message, error_msg]]
        return "", new_history

def add_document_fn(content: str) -> str:
    """RAG 시스템에 문서 추가"""
    if not content.strip():
        return "❌ 빈 문서는 추가할 수 없습니다."
    
    try:
        doc_id = rag_system.add_document(content)
        return f"✅ 문서가 성공적으로 추가되었습니다. (ID: {doc_id})"
    except Exception as e:
        return f"❌ 문서 추가 중 오류가 발생했습니다: {str(e)}"

def export_chat_history_fn(system: str) -> str:
    """채팅 기록 내보내기"""
    return chat_history.export_history(system)

def clear_chat_history_fn(system: str) -> str:
    """채팅 기록 삭제"""
    chat_history.clear_history(system)
    return f"{system} 시스템의 채팅 기록이 삭제되었습니다."

print("✅ Gradio 인터페이스 함수 정의 완료")

## 7. Gradio 인터페이스 구성

In [ ]:
def create_gradio_interface():
    """Gradio 인터페이스 생성"""
    
    # 커스텀 CSS
    custom_css = """
    .gradio-container {
        font-family: 'Arial', sans-serif !important;
    }
    
    .chat-message {
        padding: 10px;
        margin: 5px 0;
        border-radius: 10px;
    }
    
    .user-message {
        background-color: #e3f2fd;
        text-align: right;
    }
    
    .bot-message {
        background-color: #f5f5f5;
        text-align: left;
    }
    """
    
    # 메인 인터페이스
    with gr.Blocks(
        title="🚀 Day 3: LangGraph + Gradio 통합 시스템",
        theme=gr.themes.Soft(),
        css=custom_css
    ) as demo:
        
        # 헤더
        gr.Markdown("""
        # 🚀 Day 3: LangGraph + Gradio 통합 시스템
        
        Day 1에서 파인튜닝한 EXAONE 모델과 LangGraph를 활용한 통합 AI 시스템입니다.
        
        ## 🔧 시스템 구성
        - **RAG 시스템**: 문서 검색 기반 질의응답
        - **Text2SQL 시스템**: 자연어를 SQL로 변환하여 데이터베이스 조회
        - **MCP 통합 시스템**: 다양한 외부 도구를 활용한 멀티태스킹
        """)
        
        # 탭 구성
        with gr.Tabs() as tabs:
            
            # RAG 시스템 탭
            with gr.TabItem("📚 RAG 시스템", id="rag_tab"):
                gr.Markdown("### 📚 고급 RAG (Retrieval-Augmented Generation) 시스템")
                gr.Markdown(
                    "문서를 검색하여 컨텍스트를 제공하고, Day 1 파인튜닝 모델로 답변을 생성합니다."
                )
                
                with gr.Row():
                    with gr.Column(scale=3):
                        rag_chatbot = gr.Chatbot(
                            label="RAG 대화",
                            height=400,
                            show_copy_button=True
                        )
                        rag_msg = gr.Textbox(
                            label="메시지",
                            placeholder="AI나 머신러닝에 대해 질문해보세요...",
                            lines=2
                        )
                        
                        with gr.Row():
                            rag_send_btn = gr.Button("📤 전송", variant="primary")
                            rag_clear_btn = gr.Button("🗑 대화 지우기", variant="secondary")
                    
                    with gr.Column(scale=1):
                        gr.Markdown("#### 📄 문서 관리")
                        doc_input = gr.Textbox(
                            label="새 문서 추가",
                            placeholder="새로운 문서 내용을 입력하세요...",
                            lines=5
                        )
                        add_doc_btn = gr.Button("➕ 문서 추가", variant="secondary")
                        doc_status = gr.Textbox(
                            label="상태",
                            interactive=False,
                            lines=2
                        )
                        
                        gr.Markdown("#### 💾 기록 관리")
                        rag_export_btn = gr.Button("📥 대화 내보내기")
                        rag_export_output = gr.Textbox(
                            label="내보낸 대화",
                            lines=5,
                            max_lines=10
                        )
            
            # Text2SQL 시스템 탭
            with gr.TabItem("🗄 Text2SQL 시스템", id="sql_tab"):
                gr.Markdown("### 🗄 Text-to-SQL 시스템")
                gr.Markdown(
                    "자연어 질문을 SQL 쿼리로 변환하고 데이터베이스에서 정보를 조회합니다."
                )
                
                with gr.Row():
                    with gr.Column(scale=3):
                        sql_chatbot = gr.Chatbot(
                            label="Text2SQL 대화",
                            height=400,
                            show_copy_button=True
                        )
                        sql_msg = gr.Textbox(
                            label="데이터 질문",
                            placeholder="예: 가장 비싼 상품은 무엇인가요? 전자제품 카테고리의 총 주문 수량은?",
                            lines=2
                        )
                        
                        with gr.Row():
                            sql_send_btn = gr.Button("📤 질문하기", variant="primary")
                            sql_clear_btn = gr.Button("🗑 대화 지우기", variant="secondary")
                    
                    with gr.Column(scale=1):
                        gr.Markdown("#### 🏗 데이터베이스 스키마")
                        schema_display = gr.Textbox(
                            value=text2sql_system.schema_info,
                            label="스키마 정보",
                            interactive=False,
                            lines=12,
                            max_lines=15
                        )
                        
                        gr.Markdown("#### 💾 기록 관리")
                        sql_export_btn = gr.Button("📥 대화 내보내기")
                        sql_export_output = gr.Textbox(
                            label="내보낸 대화",
                            lines=5,
                            max_lines=10
                        )
            
            # MCP 시스템 탭
            with gr.TabItem("🔧 MCP 통합 시스템", id="mcp_tab"):
                gr.Markdown("### 🔧 MCP (Model Context Protocol) 통합 시스템")
                gr.Markdown(
                    "계산기, 날씨, 검색 등 다양한 외부 도구를 자동으로 선택하여 활용합니다."
                )
                
                with gr.Row():
                    with gr.Column(scale=3):
                        mcp_chatbot = gr.Chatbot(
                            label="MCP 대화",
                            height=400,
                            show_copy_button=True
                        )
                        mcp_msg = gr.Textbox(
                            label="멀티 태스크 요청",
                            placeholder="예: 123 + 456을 계산해주고, 서울 날씨도 알려주세요",
                            lines=2
                        )
                        
                        with gr.Row():
                            mcp_send_btn = gr.Button("📤 실행하기", variant="primary")
                            mcp_clear_btn = gr.Button("🗑 대화 지우기", variant="secondary")
                    
                    with gr.Column(scale=1):
                        gr.Markdown("#### 🛠 사용 가능한 도구들")
                        tools_info = gr.Markdown(
                            """
                            **🧮 계산기**
                            - 기본 수학 연산 (+, -, *, /)
                            - 예: "123 + 456 계산해줘"
                            
                            **🌤 날씨 정보**
                            - 한국 주요 도시 날씨
                            - 예: "서울 날씨 어때?"
                            
                            **🔍 웹 검색**
                            - 키워드 기반 정보 검색
                            - 예: "인공지능 정보 검색해줘"
                            """
                        )
                        
                        gr.Markdown("#### 💾 기록 관리")
                        mcp_export_btn = gr.Button("📥 대화 내보내기")
                        mcp_export_output = gr.Textbox(
                            label="내보낸 대화",
                            lines=5,
                            max_lines=10
                        )
            
            # 시스템 정보 탭
            with gr.TabItem("ℹ️ 시스템 정보", id="info_tab"):
                gr.Markdown("""
                ### 🔧 시스템 아키텍처
                
                #### 🧠 핵심 기술 스택
                - **Day 1 파인튜닝 모델**: `ryanu/my-exaone-raft-model`
                - **LangGraph**: 복잡한 워크플로 구성
                - **Gradio**: 사용자 인터페이스
                - **ChromaDB**: 벡터 데이터베이스 (RAG)
                - **SQLite**: 관계형 데이터베이스 (Text2SQL)
                
                #### 📊 성능 특징
                - **다중 시스템 통합**: 3개의 AI 시스템을 하나의 인터페이스로
                - **실시간 처리**: 사용자 요청에 즉시 응답
                - **확장 가능성**: 새로운 도구와 기능 추가 용이
                - **사용자 친화적**: 직관적인 웹 인터페이스
                
                #### 🚀 활용 사례
                1. **교육 도구**: AI 개념 학습 및 실습
                2. **데이터 분석**: 자연어로 데이터베이스 조회
                3. **업무 자동화**: 다양한 도구를 통합한 작업 처리
                4. **연구 도구**: 정보 검색 및 문서 분석
                
                #### 🛠 기술적 구현
                - **모듈화된 설계**: 각 시스템이 독립적으로 동작
                - **오류 처리**: 견고한 예외 처리 및 복구
                - **메모리 최적화**: 효율적인 리소스 관리
                - **확장성**: 새로운 기능 추가 용이
                
                ---
                
                **💡 개발자 정보**
                - 이 시스템은 Day 1-3 통합 실습 프로젝트입니다.
                - 파인튜닝 → RAG → 고급 LangGraph 워크플로 → UI 래핑 과정을 완성했습니다.
                """)
                
                gr.Markdown("""
                ### 📈 사용 통계
                """)
                
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### RAG 시스템")
                        rag_docs_count = gr.Number(
                            value=len(rag_system.documents),
                            label="등록된 문서 수",
                            interactive=False
                        )
                    
                    with gr.Column():
                        gr.Markdown("#### Text2SQL 시스템")
                        sql_tables_count = gr.Number(
                            value=2,  # products, orders
                            label="데이터베이스 테이블 수",
                            interactive=False
                        )
                    
                    with gr.Column():
                        gr.Markdown("#### MCP 시스템")
                        mcp_tools_count = gr.Number(
                            value=len(mcp_system.tools),
                            label="사용 가능한 도구 수",
                            interactive=False
                        )
        
        # 이벤트 핸들러 연결
        
        # RAG 시스템 이벤트
        rag_msg.submit(
            rag_chat_fn,
            inputs=[rag_msg, rag_chatbot],
            outputs=[rag_msg, rag_chatbot]
        )
        rag_send_btn.click(
            rag_chat_fn,
            inputs=[rag_msg, rag_chatbot],
            outputs=[rag_msg, rag_chatbot]
        )
        rag_clear_btn.click(
            lambda: ([], "RAG 대화 기록이 지워졌습니다."),
            outputs=[rag_chatbot, doc_status]
        )
        add_doc_btn.click(
            add_document_fn,
            inputs=[doc_input],
            outputs=[doc_status]
        )
        rag_export_btn.click(
            lambda: export_chat_history_fn("rag"),
            outputs=[rag_export_output]
        )
        
        # Text2SQL 시스템 이벤트
        sql_msg.submit(
            text2sql_chat_fn,
            inputs=[sql_msg, sql_chatbot],
            outputs=[sql_msg, sql_chatbot]
        )
        sql_send_btn.click(
            text2sql_chat_fn,
            inputs=[sql_msg, sql_chatbot],
            outputs=[sql_msg, sql_chatbot]
        )
        sql_clear_btn.click(
            lambda: [],
            outputs=[sql_chatbot]
        )
        sql_export_btn.click(
            lambda: export_chat_history_fn("text2sql"),
            outputs=[sql_export_output]
        )
        
        # MCP 시스템 이벤트
        mcp_msg.submit(
            mcp_chat_fn,
            inputs=[mcp_msg, mcp_chatbot],
            outputs=[mcp_msg, mcp_chatbot]
        )
        mcp_send_btn.click(
            mcp_chat_fn,
            inputs=[mcp_msg, mcp_chatbot],
            outputs=[mcp_msg, mcp_chatbot]
        )
        mcp_clear_btn.click(
            lambda: [],
            outputs=[mcp_chatbot]
        )
        mcp_export_btn.click(
            lambda: export_chat_history_fn("mcp"),
            outputs=[mcp_export_output]
        )
    
    return demo

print("✅ Gradio 인터페이스 구성 완료")

## 8. 시스템 실행

In [ ]:
if __name__ == "__main__":
    print("🚀 Day 3: LangGraph + Gradio 통합 시스템 시작")
    print("=" * 60)
    
    # 시스템 상태 확인
    print(f"📚 RAG 시스템: {len(rag_system.documents)}개 문서 로드됨")
    print(f"🗄 Text2SQL 시스템: 데이터베이스 연결됨")
    print(f"🔧 MCP 시스템: {len(mcp_system.tools)}개 도구 사용 가능")
    print(f"🧠 LLM 모델: {llm.model_name}")
    
    # Gradio 인터페이스 생성 및 실행
    demo = create_gradio_interface()
    
    print("\n🌐 웹 인터페이스를 실행합니다...")
    print("브라우저에서 자동으로 열립니다.")
    print("\n💡 사용 방법:")
    print("1. 📚 RAG 탭: AI 관련 질문을 하고 문서를 추가해보세요")
    print("2. 🗄 Text2SQL 탭: 자연어로 데이터베이스를 조회해보세요")
    print("3. 🔧 MCP 탭: 계산, 날씨, 검색 등을 요청해보세요")
    print("4. ℹ️ 시스템 정보 탭: 시스템 구조와 통계를 확인하세요")
    
    # 인터페이스 실행
    demo.launch(
        server_name="0.0.0.0",  # 모든 IP에서 접근 가능
        server_port=7860,       # 포트 설정
        share=False,            # 공개 링크 생성하지 않음 (로컬 사용)
        inbrowser=True,         # 브라우저에서 자동으로 열기
        show_tips=True,         # 팁 표시
        enable_queue=True       # 큐 시스템 활성화
    )

## 9. 고급 기능: 파일 업로드 및 내보내기

In [ ]:
# 파일 처리 함수들
def process_uploaded_file(file_path: str) -> str:
    """업로드된 파일 처리"""
    if not file_path:
        return "파일이 선택되지 않았습니다."
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 문서를 RAG 시스템에 추가
        doc_id = rag_system.add_document(content, {"source": "uploaded_file"})
        
        return f"✅ 파일이 성공적으로 처리되었습니다. (문서 ID: {doc_id})\n내용 길이: {len(content)} 글자"
        
    except Exception as e:
        return f"❌ 파일 처리 중 오류가 발생했습니다: {str(e)}"

def download_chat_history(system: str):
    """채팅 기록을 파일로 다운로드"""
    history_text = chat_history.export_history(system)
    
    # 임시 파일 생성
    temp_file = tempfile.NamedTemporaryFile(
        mode='w',
        encoding='utf-8',
        suffix=f'_{system}_history.txt',
        delete=False
    )
    
    temp_file.write(history_text)
    temp_file.close()
    
    return temp_file.name

print("✅ 파일 처리 함수 정의 완료")

## 10. 시스템 모니터링 및 분석

In [ ]:
class SystemMonitor:
    """시스템 모니터링 클래스"""
    
    def __init__(self):
        self.start_time = datetime.now()
        self.stats = {
            "rag_queries": 0,
            "sql_queries": 0,
            "mcp_queries": 0,
            "total_errors": 0,
            "documents_added": 0
        }
    
    def log_query(self, system: str, success: bool = True):
        """쿼리 로깅"""
        if system in ["rag", "sql", "mcp"]:
            self.stats[f"{system}_queries"] += 1
        
        if not success:
            self.stats["total_errors"] += 1
    
    def log_document_added(self):
        """문서 추가 로깅"""
        self.stats["documents_added"] += 1
    
    def get_stats(self) -> Dict[str, Any]:
        """통계 반환"""
        runtime = datetime.now() - self.start_time
        
        return {
            **self.stats,
            "runtime_minutes": runtime.total_seconds() / 60,
            "total_queries": sum([
                self.stats["rag_queries"],
                self.stats["sql_queries"],
                self.stats["mcp_queries"]
            ])
        }
    
    def get_performance_report(self) -> str:
        """성능 리포트 생성"""
        stats = self.get_stats()
        
        report = f"""🔍 시스템 성능 리포트
==========================================

⏱️ 실행 시간: {stats['runtime_minutes']:.1f}분
📊 총 쿼리 수: {stats['total_queries']}개

📚 RAG 시스템: {stats['rag_queries']}개 쿼리
🗄️ Text2SQL 시스템: {stats['sql_queries']}개 쿼리  
🔧 MCP 시스템: {stats['mcp_queries']}개 쿼리

📄 추가된 문서: {stats['documents_added']}개
❌ 총 오류: {stats['total_errors']}개

💡 성공률: {(stats['total_queries'] - stats['total_errors']) / max(stats['total_queries'], 1) * 100:.1f}%
"""
        
        return report

# 글로벌 모니터링 인스턴스
system_monitor = SystemMonitor()
print("✅ 시스템 모니터링 초기화 완료")

## 11. 실습 완료 및 다음 단계

In [ ]:
print("""🎓 Day 3 - Gradio UI 통합 실습 완료!

🏆 달성한 목표:
✅ Day 1 파인튜닝 모델을 모든 시스템에 통합
✅ 고급 RAG 시스템 구현 및 UI 연동
✅ Text2SQL 시스템 구현 및 UI 연동  
✅ MCP 통합 시스템 구현 및 UI 연동
✅ 사용자 친화적인 웹 인터페이스 구축
✅ 실시간 채팅 및 기록 관리 기능
✅ 시스템 모니터링 및 성능 분석

🌟 핵심 학습 내용:
1. **모델 통합**: 파인튜닝 모델의 실제 활용
2. **워크플로 설계**: LangGraph를 통한 복잡한 AI 워크플로
3. **UI 설계**: Gradio를 통한 전문적인 웹 인터페이스
4. **시스템 통합**: 여러 AI 시스템의 효과적인 통합
5. **사용자 경험**: 직관적이고 실용적인 UX 설계

🚀 실제 활용 가능성:
- 기업용 AI 어시스턴트 시스템
- 교육용 AI 플랫폼
- 데이터 분석 도구
- 연구용 정보 검색 시스템
- 업무 자동화 플랫폼

💡 확장 아이디어:
- 음성 입력/출력 추가
- 다국어 지원
- 더 많은 외부 API 통합
- 사용자 인증 및 개인화
- 클라우드 배포 및 스케일링

🎯 3일차 여정 완료:
Day 1: 파인튜닝 → Day 2: RAG → Day 3: LangGraph + UI
완전한 AI 시스템 구축 여정을 마쳤습니다!
""")

# 최종 시스템 상태 체크
print("\n📊 최종 시스템 상태:")
print(f"- RAG 문서: {len(rag_system.documents)}개")
print(f"- SQL 테이블: 2개 (products, orders)")
print(f"- MCP 도구: {len(mcp_system.tools)}개")
print(f"- LLM 모델: {llm.model_name}")
print("\n🎉 모든 시스템이 정상 작동 중입니다!")

print("\n🚀 이제 위의 셀을 실행하여 웹 인터페이스를 시작하세요!")
print("   실행 후 브라우저에서 http://localhost:7860 으로 접속할 수 있습니다.")